In [1]:
%cd /Users/trancatkhanh/Downloads/GAME
from base.Splendor_v3.env import *


/Users/trancatkhanh/Downloads/GAME


# Homework 3:
**1 agent tổng quán sử dụng bộ feature và bias có tỷ lệ thắng trên 60%**

# Bước 1 :Lấy bias của các action

In [2]:
from base.Splendor_v3.env import *
import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 10000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 1000,bias1,0)
print(win)

[0.96463848 0.94899909 1.         0.99217976 0.89586445 0.90616839
 0.91950055 0.92973951 0.88693796 0.9118531  0.92083089 0.9239534
 0.93158299 0.89188172 0.91427949]
525


# Bước 2: Ghép các feature với nhau 

In [ ]:
def get_feature5(i):
  #State old là state Hiện tại 
  #State new là state Mới 
  old = np.load('old'+f'{i}'+'.npy')
  new = np.load('new'+f'{i}'+'.npy')
  rate = 0 
  while rate<0.2 :

    # state mà có cả Value của old and new (1,0)
    o = (new>old)
    o= 1*o
    i = np.zeros((old.shape[0],1))
    for u in range(old.shape[0]) :
        if sum(o[u])>0 :
            # print(o[u])
            i[u] = 1
    #  a matrix with random value
    l = np.random.uniform(-1,1,(161,5))
    for j in range(getStateSize()):
      for vl in range(0,5):
        l[j][vl] = np.random.choice([-1,0,1], 1, p=[0.05, 0.75, 0.2])

        # if l[j][vl] > np.random.randn(1):
        #     l[j][vl] = 1
        # elif l[j][vl] < np.random.randn(1):
        #     l[j][vl] =  0
        # else:
        #     l[j][vl] = -1
    #state mà kích hoạt (1,0)
    
    k = 1*((new@l)>=0)
    # state mà kích hoạt và có reward (1,0)
    y = np.zeros((old.shape[0],1))
    for p in range(old.shape[0]):
        if sum(k[p]) >= 1 and sum(i[p])>= 1:
            y[p] = 1
        else:
            y[p] = 0
      
    rate = (np.sum(y)/np.sum(k))

  return l.reshape(5,161)
matrix_feature_check_win_rate = []
for k in range(0,15): 
  matrix  = get_feature5(k)
  matrix_feature = []
  matrix_feature.append(matrix)
  perx=[np.zeros((5,getStateSize())),np.zeros((1,5))]
  for i in range(0,5):
      for j in range(getStateSize()):
          perx[0][i][j] += matrix_feature[0][0][j]
  @njit
  def p0(state,per):
      actions = getValidActions(state)
      matran1 = per[0]@state    
      matran1 = 1*(matran1 >= 0)
      actions = np.where(actions == 1)[0]
      if np.sum(matran1) == 5 and k in actions :
          action = k
      else:
          actions = np.where(actions == 1)[0]
          action = actions[np.random.randint(len(actions))]
      return action,per
  win, x = numba_main_2(p0, 1000,perx,0)
  while  win<350 :
    matrix  = get_feature5(k)
    matrix_feature = []
    matrix_feature.append(matrix)
    perx=[np.zeros((5,getStateSize())),np.zeros((1,5))]
    for i in range(0,5):
        for j in range(getStateSize()):
            perx[0][i][j] += matrix_feature[0][0][j]
    win, x = numba_main_2(p0, 1000,perx,0)
    print('số trận thắng của action '+f'{k}',win)

  print('số trận thắng của action '+f'{k}',win)

  matrix_feature_check_win_rate.append(matrix)



In [290]:
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = per[0][0:5]@state    
    matran1 = 1*(matran1 >= 0)
    actions = np.where(actions == 1)[0]

    if np.sum(matran1) == 5 and 0 in actions :
        action = 0
    else:
        action = actions[np.random.randint(len(actions))]
    return action,per
win, x = numba_main_2(p0, 1000,perx,0)
print(win)

246


In [ ]:
perx=[np.zeros((75,getStateSize())),np.zeros((getActionSize(),75))]
for vk in range(len(matrix_feature_check_win_rate)):
  for i in range(0,5):
      for j in range(getStateSize()):
          perx[0][vk*5+i][j] += matrix_feature_check_win_rate[vk][i][j]
for i in range(0,15):
    for j in range(5):
        perx[1][i][i*5+j] = 1
        
from base.Splendor_v3.env import *
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = per[0]@state
    matran1 = 1*(matran1 >= 0)
    matran2 = per[1]@matran1
    matran2 *= actions
    print(matran2)
    if len(np.where(matran2 == matran2[np.argmax(matran2)])[0])>1 and np.max(matran2) == 5:
        matran2 += bias_greedy
        action = np.argmax(matran2)
    else:
        action = np.argmax(matran2)
    return action,per
win, x = numba_main_2(p0, 1,perx,0)

In [252]:
print(win)

4991


In [223]:
perx=[np.zeros((75,getStateSize())),np.zeros((getActionSize(),75)),np.zeros((2,1))]
for i in range(0,75):
    for j in range(getStateSize()):
        perx[0][i][j] = np.random.choice([-1,0,1], 1, p=[0.1, 0.7, 0.2])

for i in range(0,getActionSize()):
    for j in range(5):
        perx[1][i][i*5+j] = 1

# for i in range(0,getActionSize()):
#     for j in range(75):
#         if perx[1][i][j] > np.random.randn(1):
#             perx[1][i][j] = 1
#         else:
#             perx[1][i][j] =  0
print(perx[2][0])

[0.]


In [238]:
from base.Splendor_v3.env import *
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = per[0]@state
    # if per[2][0][1] >10  and per[2][0][0]/per[2][0][1] > 0.6:
    #     perx = [np.random.randn(5,getStateSize())]
    #     for i in range(0,5):
    #         for j in range(getStateSize()):
    #             if per[0][i][j] > np.random.randn(1):
    #                 per[0][i][j] = 1
    #             elif per[0][i][j] < np.random.randn(1):
    #                 per[0][i][j] =  0
    #             else:
    #                 per[0][i][j] = -1
    #     per[2][0][0] = 0
    #     per[2][0][1] = 0

    for i in range(len(matran1)):
        matran1[i] = int(matran1[i] >= 0)
    matran2 = per[1]@matran1
    matran2 *= actions
    if len(np.where(matran2 == matran2[np.argmax(matran2)])[0])>1 and np.max(matran2) == 5:
        matran2 += bias_greedy
        action = np.argmax(matran2)
    else:
        action = np.argmax(matran2)
    # if getReward(state)==1:
    #     per[2][0][0] += 1
    #     per[2][0][1] += 1
    # elif getReward(state)==0:
    #     per[2][0][1] += 1
    return action,per
win, x = numba_main_2(p0, 100000,perx,0)

IndexError: getitem out of range

In [ ]:
print(win)

4839


In [194]:
print(np.random.randn(1))

[1.14614932]
